In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
DATA_PATH = './data/'
LIGHTCURVES_PATH = DATA_PATH + 'lightcurves/'
FEATURES_PATH = DATA_PATH + 'features/'

In [3]:
import numpy as np
import pandas as pd
import measurements, extract
import matplotlib.pyplot as plt
import inputs2
from log_progress import log_progress

In [4]:
def unique_ids_list(df_lcs):
    return df_lcs.index.get_level_values('ID').unique().format()

def print_num_ids_shape(df_lcs):
    unique_ids = unique_ids_list(df_lcs)
    print('Num IDs: {}  Shape: {}'.format(len(unique_ids), df_lcs.shape))

#### Import

Import __supernova__ lightcurves

In [5]:
filename = 'sns_clean.pickle'
indir = LIGHTCURVES_PATH; filepath = indir + filename
sns = pd.read_pickle(filepath)
print_num_ids_shape(sns)

Num IDs: 5000  Shape: (186510, 4)


In [6]:
(sns.index.get_level_values("ID").unique())

Index(['SN_0', 'SN_1', 'SN_2', 'SN_3', 'SN_4', 'SN_5', 'SN_6', 'SN_7', 'SN_8',
       'SN_9',
       ...
       'SN_4994', 'SN_4995', 'SN_4996', 'SN_4997', 'SN_4998', 'SN_4999',
       'SN_5000', 'SN_5001', 'SN_5002', 'SN_5003'],
      dtype='object', name='ID', length=5000)

Import __non-supernovas__ light curves

In [6]:
filename = 'not_sns_clean.pickle'
indir = LIGHTCURVES_PATH; filepath = indir + filename
nsns = pd.read_pickle(filepath)
print_num_ids_shape(nsns)

Num IDs: 172189  Shape: (8580130, 4)


Import __measured supernova___ lightcurves

In [7]:
filename = 'measured_sns_clean.pickle'
indir = LIGHTCURVES_PATH; filepath = indir + filename
msns = pd.read_pickle(filepath)
print_num_ids_shape(msns)

Num IDs: 155  Shape: (8604, 4)


In [8]:
msns.head()


MJD        FLUX  FLUX_ERROR BANDPASS
ID        observation_id                                          
MSN_10502 0               53169.0  258.411133    0.353578        z
          1               53198.0   84.515526    0.065914        r
          2               53193.0  417.638702    0.081262        i
          3               53178.0  175.297058    0.045678        r
          4               53178.0  458.938141    0.156932        i

In [9]:
filename = "diego_t_clean.pickle"
indir = LIGHTCURVES_PATH; filepath = indir + filename
nt = pd.read_pickle(filepath)
print_num_ids_shape(nt)

Num IDs: 4869  Shape: (440469, 3)


In [11]:
nt.tail()

Mag    Magerr           MJD
ID                        observation_id                                 
TranID1209190181234125108 451469          18.1952  0.123607  56189.314816
                          451470          18.2097  0.124706  56189.322996
                          451471          18.5116  0.138816  56205.285887
                          451472          18.3150  0.128915  56205.292631
                          451473          18.6342  0.152335  56218.237507

### Add class and set BANDPASS as index


__SN__

In [9]:
sns['class'] = 'sn'
msns['class'] = 'sn'

__Non-SN__

In [10]:
nsns['class'] = 'non-sn'

In [11]:
sns = sns.set_index(['BANDPASS'], append=True)

In [12]:
nsns = nsns.set_index(['BANDPASS'], append=True)

In [13]:
msns= msns.set_index(['BANDPASS'],append=True)

In [14]:
msns.head()

MJD        FLUX  FLUX_ERROR class
ID        observation_id BANDPASS                                       
MSN_10502 0              z         53169.0  258.411133    0.353578    sn
          1              r         53198.0   84.515526    0.065914    sn
          2              i         53193.0  417.638702    0.081262    sn
          3              r         53178.0  175.297058    0.045678    sn
          4              i         53178.0  458.938141    0.156932    sn

#### Filter

In [ ]:
sns.index.get_values

In [18]:
sns.head()

MJD      FLUX  FLUX_ERROR class
ID   observation_id BANDPASS                                       
SN_0 0              i         53094.543   18.7260       2.064    sn
     1              r         53094.586    7.3375       1.673    sn
     2              z         53094.613    9.8390       9.255    sn
     3              i         53109.492  125.7100       3.098    sn
     4              r         53109.539  110.4600       2.639    sn

# Make columns index again and then run the filter

In [19]:
def filter_light_curves(sns, min_obs):
#     df_count = df_lcs.groupby(['ID','BANDPASS'], as_index=True).count()
#     df_count['ObsCount'] = df_count['FLUX']
#     df_count = df_count[['ObsCount']]
# #     df_count.head()
#     df_lcs_with_counts = df_lcs.join(df_count, how='inner')
#     # Remove objects with less than min_obs
#     df_filtered = df_lcs_with_counts[df_lcs_with_counts.ObsCount >= min_obs]
    
    df_count = sns.groupby(['ID','BANDPASS'], as_index=True).count()
    df_count['ObsCount'] = df_count['FLUX']
    df_count = df_count[['ObsCount']]

    sns = sns.reset_index(level=['ID', 'BANDPASS'])
    df_count = df_count.reset_index(level=['ID', 'BANDPASS'])

    df_lcs_with_counts = sns.merge(df_count,on=['ID','BANDPASS'])

    df_lcs_with_counts.head()

    df_lcs_with_counts.index.name = 'observation_id'
    df_lcs_with_counts = df_lcs_with_counts.set_index(['ID'], append=True)
    df_lcs_with_counts = df_lcs_with_counts.set_index(['BANDPASS'], append=True)
    df_lcs_with_counts = df_lcs_with_counts.reorder_levels(['ID', 'observation_id','BANDPASS'])
#     # Remove ObsCount
    df_filtered = df_lcs_with_counts[df_lcs_with_counts.ObsCount >= min_obs]
    df_filtered = df_filtered.drop(['ObsCount'], axis=1)
# 

    return df_filtered

def merge_multi(self, df, on):
    return self.reset_index().join(df,on=on,how=inner).set_index(self.index.names)

def sample(df_lcs, num_samples):
    # Set random seed
    np.random.seed(42)
    # Sample non-transient subset of same size as transients
    IDs = np.random.choice(unique_ids_list(df_lcs), size=num_samples, replace=False)
#     print(IDs); return
    df_sampled = df_nont.loc[IDs]
    return df_sampled

Filter __supernova__ light curves

In [20]:
df_sn_5 = filter_light_curves(sns, 5)
print_num_ids_shape(df_sn_5)

del sns

Num IDs: 5000  Shape: (179730, 4)


In [21]:
df_sn_5.head()

MJD     FLUX  FLUX_ERROR class
ID   observation_id BANDPASS                                      
SN_0 0              i         53094.543   18.726       2.064    sn
     1              i         53109.492  125.710       3.098    sn
     2              i         53112.527  139.860       4.138    sn
     3              i         53116.500  151.710       3.352    sn
     4              i         53117.441  156.310       4.463    sn

Filter __non-supernova__ lightcurves

In [16]:
df_nsns_5 = filter_light_curves(nsns, 5)
print_num_ids_shape(df_nsns_5)

del nsns

Num IDs: 161855  Shape: (7806031, 4)


In [35]:
df_msns_5 = filter_light_curves(msns,5)
print_num_ids_shape(df_msns_5)
del msns

Num IDs: 153  Shape: (8236, 4)


#### Oversample - we dont need this - set copies to 0 

In [23]:
def oversample(df_lcs, copies=0):
    df_oversample = df_lcs.copy()
    df_oversample['copy_num'] = 0
    for i in range(1, copies+1):
        df_temp = df_lcs.copy()
        df_temp['copy_num'] = i
        df_temp['Mag'] = np.random.normal(df_lcs.Mag, df_lcs.Magerr)
        df_oversample = df_oversample.append(df_temp)
        
    df_oversample = df_oversample.set_index(['copy_num'], append=True)
    return df_oversample

Oversample __supernovas__ light curves

In [24]:
df_sns_0_os = oversample(df_sn_5, 0)
print_num_ids_shape(df_sns_0_os)

del df_sn_5

Num IDs: 5000  Shape: (179730, 4)


Oversample __non-supernovas__ light curves

In [19]:
df_nsns_0_os = oversample(df_nsns_5, 0)
print_num_ids_shape(df_nsns_0_os)

del df_nsns_5

Num IDs: 161855  Shape: (7806031, 4)


Overssample __measured supernova__ light curves

In [37]:
df_msns_0_os = oversample(df_msns_5, 0)
print_num_ids_shape(df_msns_0_os)

del df_msns_5

Num IDs: 153  Shape: (8236, 4)


#### Feature Extraction

In [25]:
def extract_features(df_sns_0_os):
    num_copy_list = df_sns_0_os.index.get_level_values('copy_num').unique()    
    num_copies = len(num_copy_list)

    # Extract IDs list
    unique_ids_list = df_sns_0_os.index.get_level_values('ID').unique()
    num_ids = len(unique_ids_list)

    # Extract bandpass list
    unique_bandpass_list = df_sns_0_os.index.get_level_values('BANDPASS').unique()
    num_bandpass = len(unique_bandpass_list)

    # Create empty feature dict
    feats_dict = extract.feature_dict(30)
    feats_dict['ObsCount'] = []
    feats_dict['Class'] = []

    # Add 'ID', 'copy_num' and 'BANDPASS' index lists
    index_id_list = []
    index_copy_num_list = []
    index_bandpass_list =  []

    num_objects = num_ids*num_copies*num_bandpass
    for num_copy in num_copy_list:
        for i, obj_id in enumerate(unique_ids_list):
            for j, bandpass_id in enumerate(unique_bandpass_list):

                # Print status
                current_object_i = (num_copy+1)*i + j
                if(current_object_i%int(num_objects/10) == 0):
                    print(current_object_i, '/', num_objects)

                # Get current object light curve
                df_object = df_sns_0_os.loc[obj_id,:,bandpass_id,num_copy]
                if(len(df_object)==0): 
                    continue

                # Get features
                obj_feats = extract.features(df_object, feats_dict)

                # Append features
                for k,v in obj_feats.items():
                    feats_dict[k].append(obj_feats[k])

                # Append Indexes
                index_id_list.append(obj_id)
                index_copy_num_list.append(num_copy)
                index_bandpass_list.append(bandpass_id)

                # Append class and obs_count
                assert(len(df_object['class'].unique()) == 1)
                feats_dict['Class'].append(df_object['class'].unique()[0])
                feats_dict['ObsCount'].append(df_object.shape[0])

    # Create feature dataframe
    df_feats = pd.DataFrame(feats_dict).set_index([index_id_list,index_copy_num_list,index_bandpass_list])
    df_feats.index.names = ['ID', 'copy_num','BANDPASS']
    return df_feats

def save_features(df_feats, obj_type,filename):
    outdir = FEATURES_PATH
    filename_raw = '{}.pickle'
    filename = filename_raw.format(filename)
    assert(df_feats.shape[1]==32) # 30 + ['num_obs'+'class']
    df_feats.to_pickle(outdir + filename)

#### Generate Features

In [26]:
def generate_features(df_all, sn, min_obs,output_filename):
    obj_type = 'SN' if sn else 'NSN'
    
    # Generate features based on light curves
    df_feats = extract_features(df_all)
    
    # Save features
    save_features(df_feats, obj_type,output_filename)
    
    # Log Finished
    print('Finished task type={} obs={}'.format(obj_type, min_obs) )
    
    return df_feats

Generate features __transient__ light curves

In [27]:
sns_feats = generate_features(df_sns_0_os, sn=True, min_obs=5,output_filename="SN")

0 / 20000
2000 / 20000
2000 / 20000
2000 / 20000
2000 / 20000
4000 / 20000
4000 / 20000
4000 / 20000
4000 / 20000
Finished task type=SN obs=5


In [78]:
nsns_feats = generate_features(df_nsns_0_os, sn=False, min_obs=5,output_filename="NSN")

0 / 647420


/home/mauro/Documents/U/monitoriaInv/monInv/measurements.py:72: RuntimeWarning: invalid value encountered in double_scalars
  weighted_mean = (ss_mag / ss_magerr).sum() /  (1./ss_magerr).sum()
/home/mauro/Documents/U/monitoriaInv/monInv/measurements.py:110: RuntimeWarning: invalid value encountered in double_scalars
  return sigmas.abs().mean() / np.sqrt(sigmas.pow(2.).mean())


64742 / 647420
64742 / 647420
64742 / 647420
64742 / 647420
129484 / 647420
129484 / 647420
129484 / 647420
129484 / 647420
Finished task type=NSN obs=5


In [25]:
msns_feats = generate_features(df_msns_0_os,sn=True,min_obs=5,output_filename="MSN")

0 / 612
61 / 612
61 / 612
61 / 612
61 / 612
122 / 612
122 / 612
122 / 612
122 / 612
Finished task type=SN obs=5
